In [ ]:
from pymongo.mongo_client import MongoClient
import json
occupation = 'politician'
uri = "mongodb://localhost:27017/"
client = MongoClient(uri)
db = client['wikidata_json']
collection = db[occupation]
gt_user = "<YOUR GT USERNAME>"
parent_dir = "/home/hice1/"+gt_user+"/scratch/nl_data/English_Labelled_Wikidata_" + occupation + "/"

In [9]:
import os
from pymongo import UpdateOne
for filename in os.listdir(parent_dir):
    file_path = os.path.join(parent_dir, filename)  
    if os.path.isfile(file_path):        
        with open(file_path, 'r') as f:
            data = json.load(f)
            print("Loaded JSON file for Part " + str(file_path))
            # Convert each entity to a format MongoDB can store
            documents = [{**{"_id": key}, **value} for key, value in data.items()]
            operations = [
                UpdateOne({"_id": doc["_id"]}, {"$set": doc}, upsert=True)
                for doc in documents
            ]
            
            # Perform bulk write
            if operations:  # Ensure there are operations to execute
                collection.bulk_write(operations)
                print("Added or updated data from Part " + str(file_path) + " to Database")
            print("Added data from Part " + str(file_path) + " to Database")
print("Creating Index based on Given name")
# Index 'given name' for fast search
collection.create_index("given name")
collection.create_index("Commons category")

Loaded JSON file for Part /home/hice1/tkaple3/scratch/nl_data/English_Labelled_Wikidata_politician/en_labelled_part12.json
Added or updated data from Part /home/hice1/tkaple3/scratch/nl_data/English_Labelled_Wikidata_politician/en_labelled_part12.json to Database
Added data from Part /home/hice1/tkaple3/scratch/nl_data/English_Labelled_Wikidata_politician/en_labelled_part12.json to Database
Loaded JSON file for Part /home/hice1/tkaple3/scratch/nl_data/English_Labelled_Wikidata_politician/en_labelled_part9.json
Added or updated data from Part /home/hice1/tkaple3/scratch/nl_data/English_Labelled_Wikidata_politician/en_labelled_part9.json to Database
Added data from Part /home/hice1/tkaple3/scratch/nl_data/English_Labelled_Wikidata_politician/en_labelled_part9.json to Database
Loaded JSON file for Part /home/hice1/tkaple3/scratch/nl_data/English_Labelled_Wikidata_politician/en_labelled_part3242.json
Added or updated data from Part /home/hice1/tkaple3/scratch/nl_data/English_Labelled_Wikid

'Commons category_1'

In [28]:
results = collection.find({
    "$and": [
        {"given name": "John"},
        {"family name": "Wood"}
    ]
})
final_results = []
for result in results:
    final_results.append(result)
    print(result)
    print("\n")

{'_id': 'Q64485', 'Find a Grave memorial ID': ['7591248'], 'sex or gender': ['male'], 'country of citizenship': ['United States of America'], 'occupation': ['politician', 'judge'], 'position held': ['member of the United States House of Representatives'], 'date of birth': [{'time': '+1784-02-16T00:00:00Z', 'timezone': 0, 'before': 0, 'after': 0, 'precision': 11, 'calendarmodel': 'http://www.wikidata.org/entity/Q1985727'}], 'date of death': [{'time': '+1874-05-20T00:00:00Z', 'timezone': 0, 'before': 0, 'after': 0, 'precision': 11, 'calendarmodel': 'http://www.wikidata.org/entity/Q1985727'}], 'instance of': ['human'], 'given name': ['John'], 'member of political party': ['Democratic Party'], 'US Congress Bio ID': ['W000697'], 'place of birth': ['Clarkstown'], 'Freebase ID': ['/m/02rksj5'], 'languages spoken, written or signed': ['English'], 'work location': ['Washington, D.C.'], 'name in native language': [{'text': 'John J. Wood', 'language': 'en'}], 'place of death': ['New City'], 'fami

In [29]:
final_results[0]

{'_id': 'Q64485',
 'Find a Grave memorial ID': ['7591248'],
 'sex or gender': ['male'],
 'country of citizenship': ['United States of America'],
 'occupation': ['politician', 'judge'],
 'position held': ['member of the United States House of Representatives'],
 'date of birth': [{'time': '+1784-02-16T00:00:00Z',
   'timezone': 0,
   'before': 0,
   'after': 0,
   'precision': 11,
   'calendarmodel': 'http://www.wikidata.org/entity/Q1985727'}],
 'date of death': [{'time': '+1874-05-20T00:00:00Z',
   'timezone': 0,
   'before': 0,
   'after': 0,
   'precision': 11,
   'calendarmodel': 'http://www.wikidata.org/entity/Q1985727'}],
 'instance of': ['human'],
 'given name': ['John'],
 'member of political party': ['Democratic Party'],
 'US Congress Bio ID': ['W000697'],
 'place of birth': ['Clarkstown'],
 'Freebase ID': ['/m/02rksj5'],
 'languages spoken, written or signed': ['English'],
 'work location': ['Washington, D.C.'],
 'name in native language': [{'text': 'John J. Wood', 'language':

In [32]:
import re
search_res_cleaned = []
pattern = r'\bID\b'
for res in final_results:
    cleaned_res = {}
    for key, value in res.items():
        if not re.search(pattern, key):
            cleaned_res[key] = value
    search_res_cleaned.append(cleaned_res)
search_res_cleaned

[{'_id': 'Q64485',
  'sex or gender': ['male'],
  'country of citizenship': ['United States of America'],
  'occupation': ['politician', 'judge'],
  'position held': ['member of the United States House of Representatives'],
  'date of birth': [{'time': '+1784-02-16T00:00:00Z',
    'timezone': 0,
    'before': 0,
    'after': 0,
    'precision': 11,
    'calendarmodel': 'http://www.wikidata.org/entity/Q1985727'}],
  'date of death': [{'time': '+1874-05-20T00:00:00Z',
    'timezone': 0,
    'before': 0,
    'after': 0,
    'precision': 11,
    'calendarmodel': 'http://www.wikidata.org/entity/Q1985727'}],
  'instance of': ['human'],
  'given name': ['John'],
  'member of political party': ['Democratic Party'],
  'place of birth': ['Clarkstown'],
  'languages spoken, written or signed': ['English'],
  'work location': ['Washington, D.C.'],
  'name in native language': [{'text': 'John J. Wood', 'language': 'en'}],
  'place of death': ['New City'],
  'family name': ['Wood'],
  'social classi